<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Catalina Contreras
- Nombre de alumno 2:Santiago Valdivieso


### **Link de repositorio de GitHub:** `http://....`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

#Models 
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Requirement already up-to-date: plotly in c:\users\santi\anaconda3\lib\site-packages (5.4.0)


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



In [4]:
df_players.isna().sum()

Name                     0
Nationality              0
National_Position    16513
Club_Position            1
Height                   0
Weight                   0
Preffered_Foot           0
Age                      0
Work_Rate                0
Weak_foot                0
Skill_Moves              0
Ball_Control             0
Dribbling                0
Marking                  0
Sliding_Tackle           0
Standing_Tackle          0
Aggression               0
Reactions                0
Interceptions            0
Vision                   0
Composure                0
Crossing                 0
Short_Pass               0
Long_Pass                0
Acceleration             0
Speed                    0
Stamina                  0
Strength                 0
Balance                  0
Agility                  0
Jumping                  0
Heading                  0
Shot_Power               0
Finishing                0
Long_Shots               0
Curve                    0
Freekick_Accuracy        0
P

In [5]:
df_players.dropna(subset=['Club_Position'], inplace=True)

**Respuesta:**

In [6]:
### Código aquí ###
labels = df_players['National_Position'] 
labels =labels.apply (lambda row: 1 if pd.notna(row) else 0 )
labels.value_counts()
df_players1 = df_players.drop('National_Position', inplace=False, axis=1)

Menos de un 10% de los jugadores son elegidos para ser parte de la selección nacional.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [7]:
### Código aquí ###
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Name','Nationality',"Club_Position","Preffered_Foot",]),
        (' OrdinalEncoder', OrdinalEncoder(),['Work_Rate',"Weak_foot","Skill_Moves"]),
        ('StandardScaler', StandardScaler(),['Height', 'Weight', 'Age', 'Ball_Control',"Dribbling","Marking","Sliding_Tackle","Standing_Tackle",
                                             "Aggression","Reactions","Interceptions","Vision","Composure","Crossing","Short_Pass",
                                             "Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping",
                                             "Volleys","Penalties","Freekick_Accuracy","Curve","Long_Shots","Finishing","Shot_Power","Heading"])])

Categorías son transformadas a one hot encode, ordinales en ordinal encoder y el resto los pasé por standardScaler. A continuación se presentan los 3 modelos escogidos en base a la guía, Linear SVC, SVC y KNN. 

In [8]:
pipelineLSVC = Pipeline([
    ('Preprocessing', preprocessing_transformer),
     ('linearsvc', LinearSVC(random_state=0, tol=1e-05))

])
pipelineKNN = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('KNN',KNeighborsClassifier(n_neighbors=3))

])
pipelineSVC = Pipeline([
    ('Preprocessing', preprocessing_transformer),
    ('svc', SVC(gamma='auto'))

])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [9]:
### Código aquí ###
X_train, X_test, y_train, y_test = train_test_split(df_players1, labels, test_size=0.33, random_state=42)
pipelineLSVC = pipelineLSVC.fit(X_train,y_train)
y_predLSVC = pipelineLSVC.predict(X_test)

pipelineKNN = pipelineKNN.fit(X_train,y_train)
y_predKNN = pipelineKNN.predict(X_test)

pipelineSVC = pipelineSVC.fit(X_train,y_train)
y_predSVC = pipelineSVC.predict(X_test)



C:\Users\santi\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [10]:
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_predLSVC), "\n")
print(classification_report(y_test, y_predLSVC))

print("Matriz de confusión\n\n", confusion_matrix(y_test, y_predKNN), "\n")
print(classification_report(y_test, y_predKNN))


print("Matriz de confusión\n\n", confusion_matrix(y_test, y_predSVC), "\n")
print(classification_report(y_test, y_predSVC))



Matriz de confusión

 [[5395   62]
 [ 235  112]] 

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5457
           1       0.64      0.32      0.43       347

    accuracy                           0.95      5804
   macro avg       0.80      0.66      0.70      5804
weighted avg       0.94      0.95      0.94      5804

Matriz de confusión

 [[5394   63]
 [ 310   37]] 

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5457
           1       0.37      0.11      0.17       347

    accuracy                           0.94      5804
   macro avg       0.66      0.55      0.57      5804
weighted avg       0.91      0.94      0.92      5804

Matriz de confusión

 [[5457    0]
 [ 347    0]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5457
           1       0.00      0.00      0.00       347

    accuracy                 

C:\Users\santi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Respuesta**:

Se observa que linear SVC tiene el mejor resultado. Tiene un acurracy de 0.95 pero también sus otras métricas tienen buenos valores en comparacion a los otros modelos. Por otro lado, vemos que KNN es el siguiente con mejores resultados, con un acurracy de 0.95 y luego lo sigue SVC con un acurracy de 0.94. 

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [11]:
### Código aquí ###

##Como no hay que tomar en cuenta los valores Sub y Res de esta columna, los catologo como "otros"
def rename(posicion):
    if posicion in ['ST', 'CF'] :
      return "ataque"
    elif posicion in  ['RW', 'CAM', 'LW'] :
      return "central_ataque"
    elif posicion in  ['RM', 'CM', 'LM'] :
      return "central"
    elif posicion in ['RWB', 'CDM', 'LWB']:
      return "central_defensa"
    elif posicion in ['RB', 'CB', 'LB'] :
      return "defensa"
    elif posicion in ['GK']:
      return "arquero"
    else:
      return "otro"

newdf_players= df_players[df_players['Club_Position'] != "Res"].drop(['National_Position'], axis=1)
newdf_players= newdf_players[newdf_players['Club_Position'] != "Sub"]      

labels2 = newdf_players['Club_Position']
labels2 =labels2.apply (lambda row: rename(row) )
labels2.value_counts()
newdf_players.drop("Club_Position",inplace=True,axis=1)

In [12]:
### Código aquí ###
preprocessing_transformer2 = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ['Name','Nationality',"Preffered_Foot",]),
        (' OrdinalEncoder', OrdinalEncoder(),['Work_Rate',"Weak_foot","Skill_Moves"]),
        ('StandardScaler', StandardScaler(),['Height', 'Weight', 'Age', 'Ball_Control',"Dribbling","Marking","Sliding_Tackle","Standing_Tackle",
                                             "Aggression","Reactions","Interceptions","Vision","Composure","Crossing","Short_Pass",
                                             "Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping",
                                             "Volleys","Penalties","Freekick_Accuracy","Curve","Long_Shots","Finishing","Shot_Power","Heading"])])

In [13]:
pipelineLSVC2 = Pipeline([
    ('Preprocessing', preprocessing_transformer2),
     ('linearsvc', LinearSVC(random_state=0, tol=1e-05))

])
pipelineKNN2 = Pipeline([
    ('Preprocessing', preprocessing_transformer2),
    ('KNN',KNeighborsClassifier(n_neighbors=3))

])
pipelineSVC2 = Pipeline([
    ('Preprocessing', preprocessing_transformer2),
    ('svc', SVC(gamma='auto'))

])

In [14]:

X_train2, X_test2, y_train2, y_test2 = train_test_split(newdf_players, labels2, test_size=0.33, random_state=42)
pipelineLSVC2 = pipelineLSVC2.fit(X_train2,y_train2)
y_predLSVC2 = pipelineLSVC2.predict(X_test2)

pipelineKNN2 = pipelineKNN2.fit(X_train2,y_train2)
y_predKNN2 = pipelineKNN2.predict(X_test2)

pipelineSVC2 = pipelineSVC2.fit(X_train2,y_train2)
y_predSVC2 = pipelineSVC2.predict(X_test2)


C:\Users\santi\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [15]:
print("Matriz de confusión\n\n", confusion_matrix(y_test2, y_predLSVC2), "\n")
print(classification_report(y_test, y_predLSVC))

print("Matriz de confusión\n\n", confusion_matrix(y_test2, y_predKNN2), "\n")
print(classification_report(y_test, y_predKNN))


print("Matriz de confusión\n\n", confusion_matrix(y_test2, y_predSVC2), "\n")
print(classification_report(y_test, y_predSVC))

Matriz de confusión

 [[202   0   0   0   0   0   0]
 [  0  76  21   9   0   0  55]
 [  0   5 154  47   0  32  88]
 [  0   7  77  43   0   4  57]
 [  0   0   0   0   0  16  49]
 [  0   0   5   0   0 286  93]
 [  0  43  49  30   0  46 800]] 

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5457
           1       0.64      0.32      0.43       347

    accuracy                           0.95      5804
   macro avg       0.80      0.66      0.70      5804
weighted avg       0.94      0.95      0.94      5804

Matriz de confusión

 [[202   0   0   0   0   0   0]
 [  0  97  18   7   0   0  39]
 [  0  25 156  34   2  28  81]
 [  0  18  81  39   0   0  50]
 [  0   0   7   2   4  11  41]
 [  0   0  27   1  17 237 102]
 [  0  76 100  30  14  57 691]] 

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5457
           1       0.37      0.11      0.17       347

    accuracy             

C:\Users\santi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Respuesta**:
Se observa que al igual que en la primera parte según la guía los modelos a clasificar deben ser los mismos que antes, esto es por la cantidad de datos y el tipo clasificación. Se observa que pese a que sea una clasificación con más clases se obtienen resultados muy similares a los del experimento anterior, con Linear SVC teniendo los mejores resultados y métricas. 

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

In [16]:
salarios = pd.read_csv('salarios.csv')

**Respuesta**

In [17]:
salarios.rename(columns={'Player': 'Name'}, inplace=True)

In [18]:
df=pd.merge(left=df_players,
         right=salarios,
         on="Name",
         how="inner").drop(['National_Position','Unnamed: 0','Name','Nationality'], axis=1)

In [19]:
df=df.sort_values(by=['Weekly Salary'],ascending=False).iloc[15:]

In [20]:
preprocessing_transformer3 = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore'),  ["Preffered_Foot",'Club_Position']),
        ('OrdinalEncoder', OrdinalEncoder(),['Work_Rate']),
        ('StandardScaler', StandardScaler(),['Weak_foot','Skill_Moves','Height', 'Weight', 'Age', 'Ball_Control',"Dribbling","Marking","Sliding_Tackle","Standing_Tackle",
                                             "Aggression","Reactions","Interceptions","Vision","Composure","Crossing","Short_Pass",
                                             "Long_Pass","Acceleration","Speed","Stamina","Strength","Balance","Agility","Jumping",
                                             "Volleys","Penalties","Freekick_Accuracy","Curve","Long_Shots","Finishing","Shot_Power","Heading"])])

In [21]:
labels3 = df['Weekly Salary']
df.drop("Weekly Salary",inplace=True,axis=1)

In [22]:
from sklearn import linear_model
from sklearn.metrics import r2_score

In [23]:
pipelineLSVC3 = Pipeline([
    ('Preprocessing', preprocessing_transformer3),
     ('sgdregressor', linear_model.Ridge())

])

In [24]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df, labels3, test_size=0.33, random_state=34)
pipelineLSVC3 = pipelineLSVC3.fit(X_train2,y_train2)
y_predLSVC3 = pipelineLSVC3.predict(X_test2)

In [25]:
r2_score(y_test2, y_predLSVC3)

0.3557478871407943

**Respuesta**:

```
R2: Es la proporción de la varianza total de la variable explicada por la regresión. El coeficiente de determinación, también llamado R cuadrado, refleja la bondad del ajuste de un modelo a la variable que pretender explicar.


La base de datos de salarios no tenía una gran cantidad de datos, por lo que se espera que se pueda tener mejor desempeño al implementar otros modelos como SGD Regressor, el cual es mejor para cantidades mayores a 100 mil. Por otro lado, no hay características en la base que influyen considerablemente en el sueldo que recibe cada jugador, especialmente las relacionadas con el marketing. Por ejemplo, un jugador que vende una mayor cantidad de camisetas con su nombre suele ofrecerles mejores sueldos, también la cantidad de seguidores que tienen, ya que esto aumenta los ingresos de los equipos y por lo tanto ellos están dispuestos a pagarle mejor sueldo. 
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>